In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import pearsonr
import pandas as pd

import sys
sys.path.append('../../src/')
from coal_cov import *
from seg_sites_covar import CorrSegSites
from plot_utils import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['figure.facecolor'] = "w"
plt.rcParams['figure.autolayout'] = True
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker

import os
main_figdir = '../../plots/two_locus_stats/'
supp_figdir = '../../plots/supp_figs/two_locus_stats/'
os.makedirs(main_figdir, exist_ok=True)
os.makedirs(supp_figdir, exist_ok=True)

### Simulation under a model of exponential growth 

In [3]:
# NOTE : growth always starts ~ 1000 Generations in the past ... 
np.random.seed(42)

tas = [0, 100, 500]
rs = [1e-3, 1e-2, 5e-2]
rec_rate = 1e-3


total_sims = []
for t in tqdm(tas):
    for r in rs:
        cur_sim = TwoLocusSerialGrowth(ta=t, r=r, T=500, reps=50000, rec_rate=rec_rate)
        cur_sim._simulate()
        cur_sim._two_locus_branch_length()
        # Calculating the marginal variance and means
        mu_LA= np.mean(cur_sim.pair_branch_length[:,0])
        var_LA = np.var(cur_sim.pair_branch_length[:,0])
        cov_LALB = np.cov(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0,1]
        corr_LALB = pearsonr(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0]
        total_sims.append([t,r, mu_LA, var_LA, cov_LALB, corr_LALB])
            
total_sims = np.vstack(total_sims)

100%|██████████| 3/3 [03:07<00:00, 62.51s/it]


In [7]:
growth_df = pd.DataFrame(total_sims)
growth_df.columns = ['$t_a$', '$r$','$\mathbb{E}[L]$','$Var(L)$','$Cov(L_A,L_B)$','$Corr(L_A, L_B)$']
growth_df.to_csv('../../results/two_loci/growth_model.csv', index=False)
growth_df

,$t_a$,$r$,$\mathbb{E}[L]$,$Var(L)$,"$Cov(L_A,L_B)$","$Corr(L_A, L_B)$"
0,0.0,0.001,6072.365176,4.718227e+06,324213.443686,0.068411
1,0.0,0.010,1944.554757,6.568740e+04,8661.988521,0.132352
2,0.0,0.050,1253.039258,2.639417e+03,476.232940,0.179081
3,100.0,0.001,6004.463509,4.595287e+06,285275.715342,0.061896
4,100.0,0.010,1843.978938,6.512168e+04,8861.051083,0.135078
5,100.0,0.050,1153.138663,2.674536e+03,542.160411,0.203207
6,500.0,0.001,5686.825129,4.172511e+06,181323.300526,0.043535
7,500.0,0.010,1448.392250,5.945848e+04,11923.757751,0.201223
8,500.0,0.050,753.743537,2.556839e+03,905.724431,0.353543


### Simulation under a bottleneck model

In [8]:
np.random.seed(42)

tas = [0, 100, 500]
rs = [1e-1, 1e-2, 1e-3]
rec_rate = 1e-3

total_sims_bot = []
for t in tqdm(tas):
    for r in rs:
        cur_sim = TwoLocusSerialBottleneck(Ne=1e4, ta=t, Tstart=50, Tend=500, Nbot=r*1e4, reps=50000, rec_rate=rec_rate)
        cur_sim._simulate()
        cur_sim._two_locus_branch_length()
        # Calculating the marginal variance and means
        mu_LA= np.mean(cur_sim.pair_branch_length[:,0])
        var_LA = np.var(cur_sim.pair_branch_length[:,0])
        cov_LALB = np.cov(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0,1]
        corrLALB = pearsonr(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0]
        total_sims_bot.append([t, r, mu_LA, var_LA, cov_LALB, corrLALB])
        
        
total_sims_bot = np.vstack(total_sims_bot)

100%|██████████| 3/3 [02:50<00:00, 56.92s/it]


In [9]:
bottleneck_df = pd.DataFrame(total_sims_bot)
bottleneck_df.columns = ['$t_a$', '$\phi$','$\mathbb{E}[L]$','$Var(L)$','$Cov(L_A,L_B)$','$Corr(L_A,L_B)$']
bottleneck_df.to_csv('../../results/two_loci/bottleneck.csv', index=False)
bottleneck_df

,$t_a$,$\phi$,$\mathbb{E}[L]$,$Var(L)$,"$Cov(L_A,L_B)$","$Corr(L_A,L_B)$"
0,0.0,0.100,32681.304391,1.532905e+09,1.051744e+08,0.067889
1,0.0,0.010,4629.003515,3.208360e+08,6.956581e+07,0.214770
2,0.0,0.001,139.823748,1.644905e+03,1.377611e+03,0.833835
3,100.0,0.100,33148.431260,1.522341e+09,1.250056e+08,0.081292
4,100.0,0.010,5703.571589,3.976571e+08,9.865059e+07,0.248445
5,100.0,0.001,140.047018,1.625994e+03,1.499267e+03,0.922482
6,500.0,0.100,40369.613058,1.576998e+09,1.506489e+07,0.009398
7,500.0,0.010,40695.222113,1.631697e+09,1.729507e+07,0.010763
8,500.0,0.001,40245.526643,1.580981e+09,1.438117e+07,0.009079
